In [22]:
import mysql.connector
# library for data analysis
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 

import os 

In [23]:
# Membuat koneksi ke database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tourista_db'
)

In [24]:
# Mengeksekusi query untuk mengambil data dari tabel

query = "SELECT * FROM destination"
destination = pd.read_sql_query(query, conn)

query = "SELECT * FROM review_wisata"
ratings = pd.read_sql_query(query, conn)

query = "SELECT * FROM user_profile"
users = pd.read_sql_query(query, conn)

C:\Users\dewan\AppData\Local\Temp\ipykernel_12060\707316895.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  destination = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_12060\707316895.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_12060\707316895.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql_query(query, conn)


In [90]:
ratings.head(2)

,user_wisata,id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [91]:
users.head(2)

,user_wisata,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
1,2,user2,21,userTrial2@gmail.com,81326812647,"Bekasi, Jawa Barat",user2.png,-11931.0,12311.0,None,None


In [117]:
# data frame for content based filtering 
data_content = destination
data_content = data_content.drop(['time_minutes', 'coordinate'], axis=1)

data_content = data_content.rename(columns={'id': 'place_id', 'name_wisata' : 'place_name', 'category' : 'place_category'})
data_content

,place_id,place_name,description_wisata,place_category,destination_photo,city,price,rating,destination_lat,destination_long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,lala.png,Jakarta,20000.0,4.6,-6.17539,106.8272
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,lala.png,Jakarta,5000.0,4.6,-6.13764,106.8171
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,lala.png,Jakarta,270000.0,4.6,-6.12531,106.8335
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,lala.png,Jakarta,10000.0,4.5,-6.30245,106.8952
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,lala.png,Jakarta,94000.0,4.5,-6.12419,106.8391
...,...,...,...,...,...,...,...,...,...,...
432,433,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,lala.png,Surabaya,2000.0,4.4,-7.43386,112.7199
433,434,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,lala.png,Surabaya,0.0,4.6,-7.29135,112.7398
434,435,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,lala.png,Surabaya,0.0,4.4,-7.27530,112.7549
435,436,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,lala.png,Surabaya,0.0,4.6,-7.29433,112.7618


In [118]:
tfidf= TfidfVectorizer()
tfidf.fit(data_content['place_name'])

TfidfVectorizer()

In [119]:
# change data into matrix 
tfidf_matrix = tfidf.fit_transform(data_content['place_name'])
tfidf_matrix.shape

(437, 712)

In [120]:
# change vector TF-IDF into matrix 
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [121]:
df_cosine_similarity = pd.DataFrame(cosine_similarity, index = data_content['place_name'], columns = data_content['place_name'])

print('Shape : ', df_cosine_similarity.shape)

Shape :  (437, 437)


In [141]:
# recommendation function for content based filtering 

def content_rec(place_name, similarity_data = df_cosine_similarity, items = data_content, k = 5) :
  index = similarity_data.loc[:, place_name].to_numpy().argpartition(
      range(-1, -k, -1)
  )

  closest = similarity_data.columns[index[-1:-(k+2): -1]]
  closest = closest.drop(place_name, errors = 'ignore')
  result = pd.DataFrame(closest).merge(items).head(k)
  
  return result.to_dict(orient='records')

In [145]:
# test the function 
result = content_rec('Alun-Alun Kota Bandung')
result

[{'place_name': 'Alun Alun Selatan Yogyakarta',
  'place_id': 96,
  'description_wisata': 'Alun-alun Selatan atau yang sekarang lebih dikenal sebagai Alun-alun Kidul (Alkid) yaitu alun-alun yang terletak di sebelah selatan karaton Yogyakarta. Alun-alun ini berbentuk tanah lapang luas berpasir, dengan luas sekitar 160m x 160 m. Alun-alun ini dik',
  'place_category': 'Taman Hiburan',
  'destination_photo': 'lala.png',
  'city': 'Yogyakarta',
  'price': 0.0,
  'rating': 4.6,
  'destination_lat': -7.81167,
  'destination_long': 110.3632},
 {'place_name': 'Alun-alun Utara Keraton Yogyakarta',
  'place_id': 125,
  'description_wisata': 'Alun-alun utara atau dalam Bahasa Jawa disebut Alun-alun Lor merupakan salah satu land mark Kota Yogyakarta yang berupa sebuah tanah lapang yang berada di depan Keraton Yogyakarta. Disebut Alun-alun Lor karena di Kota Yogyakarta terdapat dua alun-alun yan',
  'place_category': 'Budaya',
  'destination_photo': 'lala.png',
  'city': 'Yogyakarta',
  'price': 0.